In [140]:
import math
import csv
import json

STAT_NAMES = [
    "caps", "garbage_time_caps", "hold", "ndps", "returns", "quick_returns", "nrts", "pups", 
    "keypops", "handoffs", "goodprevent", "resets", "badflaccids", "sparkedouts"
]

with open("smurfs.json") as f:
    smurfs = json.load(f)

def desmurf(player):
    player = player.replace("\\'", "'").replace("\\\\", "\\")
    if player in smurfs:
        return smurfs[player]
    return player

def row_to_dict(row):
    return {
        'match_id': int(row[0]),
        'timestamp': int(row[1]),
        'duration': int(row[2]),
        'cap_diff': int(row[3]),
        'garbage_time_cap_diff': int(row[4]),
        'players': [
            {
                'name': desmurf(row[j + 5]),
                'stats': {
                    stat_name: int(row[j * len(STAT_NAMES) + 13 + i])
                    for i, stat_name in enumerate(STAT_NAMES)
                }
            }
            for j in range(8)
        ]
    }

with open("matchups_with_stats.csv") as f:
    reader = csv.reader(f)
    next(reader)
    matches = [row_to_dict(row) for row in reader]

elo = {}
variance = {}
pred_errors = []
winner_win_prob = []

In [141]:
STARTING_VARIANCE = 1.2
VARIANCE_TO_ADD_BACK = 0.002
BASE_VARIANCE = 4.5
RED_ADVANTAGE = 0.1
PLAYER_STAT_WEIGHTS_INITIAL = {
    "caps": 0.8,
    "garbage_time_caps": -0.3,
    "hold": 1.7 / 3600,
    "ndps": -0.5,
    "returns": 0.35,
    "quick_returns": 0.0,
    "nrts": 0.3,
    "pups": 0.3,
    "keypops": 0.0,
    "handoffs": 0.2,
    "goodprevent": 0.0,
    "resets": 0.0,
    "badflaccids": -0.0,
    "sparkedouts": 0.0
}
TEAM_STAT_WEIGHTS_INITIAL = {
    "caps": 0,
    "garbage_time_caps": -0.3,
    "hold": 0.8 / 3600,
    "ndps": -0.0,
    "returns": 0.04,
    "quick_returns": 0.0,
    "nrts": 0.0,
    "pups": 0.0,
    "keypops": -0.0,
    "handoffs": 0.0,
    "goodprevent": 0.0,
    "resets": 0.0,
    "badflaccids": -0.0,
    "sparkedouts": 0.0
}
PLAYER_STAT_WEIGHTS_CONVERGED = {
    "caps": 0.05,
    "garbage_time_caps": -0.0,
    "hold": 0.0 / 3600,
    "ndps": -0.0,
    "returns": 0.0,
    "quick_returns": 0.0,
    "nrts": 0.0,
    "pups": 0.0,
    "keypops": -0.0,
    "handoffs": 0.0,
    "goodprevent": 0.0,
    "resets": 0.0,
    "badflaccids": -0.0,
    "sparkedouts": 0.0
}
TEAM_STAT_WEIGHTS_CONVERGED = {
    "caps": 0,
    "garbage_time_caps": -0.0,
    "hold": 0.4 / 3600,
    "ndps": -0.0,
    "returns": 0.02,
    "quick_returns": 0.0,
    "nrts": 0.0,
    "pups": 0.0,
    "keypops": 0.0,
    "handoffs": 0.0,
    "goodprevent": 0.0,
    "resets": 0.0,
    "badflaccids": 0.0,
    "sparkedouts": 0.0
}
DIFF_MAPPING = [0, 0.2, 0.9, 2.1, 3.1, 4.2, -4.2, -3.1, -2.1, -0.9, -0.2]
NEW_PLAYER_ELO = -1.1
NEWNESS_THRESHOLD = 0.05
NEW_PLAYER_ELO_BOOST = 0.1
DIFF_WEIGHT_INITIAL = 0.65
DIFF_WEIGHT_CONVERGED = 0.6
GARBAGE_TIME_DISCOUNT = 0.35
RELATIVE_ELO_CORRECTION = 0.05

In [142]:
def get_elo(player):
    if player not in elo:
        elo[player] = NEW_PLAYER_ELO
    return elo[player]

def get_variance(player):
    if player not in variance:
        variance[player] = STARTING_VARIANCE
    return variance[player]

def normalize_elos():
    known_elos = [elo[p] for p in elo if variance[p] <= 0.2]
    avg_elo = sum(known_elos) / max(10, len(known_elos))
    for p in elo:
        elo[p] -= avg_elo
        variance[p] = min(STARTING_VARIANCE, variance[p] + VARIANCE_TO_ADD_BACK)

In [143]:
def judge_stats_initial(players):
    scores = [
        sum([
            p['stats'][stat_name] * PLAYER_STAT_WEIGHTS_INITIAL[stat_name]
            for stat_name in STAT_NAMES
        ])
        for p in players
    ]
    team_score_diff = sum([
        sum([
            p['stats'][stat_name] * TEAM_STAT_WEIGHTS_INITIAL[stat_name]
            for stat_name in STAT_NAMES
        ])
        for p in players[:4]
    ]) - sum([
        sum([
            p['stats'][stat_name] * TEAM_STAT_WEIGHTS_INITIAL[stat_name]
            for stat_name in STAT_NAMES
        ])
        for p in players[4:]
    ])
    red_avg_score = sum(scores[:4]) / 4
    blue_avg_score = sum(scores[4:]) / 4
    red_scores = [s - red_avg_score + team_score_diff for s in scores[:4]]
    blue_scores = [s - blue_avg_score - team_score_diff for s in scores[4:]]
    return red_scores, blue_scores

def judge_stats_converged(players):
    scores = [
        sum([
            p['stats'][stat_name] * PLAYER_STAT_WEIGHTS_CONVERGED[stat_name]
            for stat_name in STAT_NAMES
        ])
        for p in players
    ]
    team_score_diff = sum([
        sum([
            p['stats'][stat_name] * TEAM_STAT_WEIGHTS_CONVERGED[stat_name]
            for stat_name in STAT_NAMES
        ])
        for p in players[:4]
    ]) - sum([
        sum([
            p['stats'][stat_name] * TEAM_STAT_WEIGHTS_CONVERGED[stat_name]
            for stat_name in STAT_NAMES
        ])
        for p in players[4:]
    ])
    red_avg_score = sum(scores[:4]) / 4
    blue_avg_score = sum(scores[4:]) / 4
    red_scores = [s - red_avg_score + team_score_diff for s in scores[:4]]
    blue_scores = [s - blue_avg_score - team_score_diff for s in scores[4:]]
    return red_scores, blue_scores

def update_rating(player, error, stat_score_initial, stat_score_converged, total_variance, elo_vs_game_avg):
    share_of_variance = variance[player] / total_variance
    newness = variance[player] / STARTING_VARIANCE
    error_weight = DIFF_WEIGHT_INITIAL * newness + DIFF_WEIGHT_CONVERGED * (1 - newness)
    stat_score = stat_score_initial * newness + stat_score_converged * (1 - newness)
    update = error_weight * error + stat_score
    update -= elo_vs_game_avg * RELATIVE_ELO_CORRECTION
    
    elo[player] += update * share_of_variance
    variance[player] *= 1 - share_of_variance
    if variance[player] > NEWNESS_THRESHOLD:
        elo[player] += NEW_PLAYER_ELO_BOOST * share_of_variance

def update_ratings(match):
    expected_score = RED_ADVANTAGE
    total_variance = BASE_VARIANCE
    
    for player in match['players'][:4]:
        expected_score += get_elo(player['name'])
        total_variance += get_variance(player['name'])
    for player in match['players'][4:]:
        expected_score -= get_elo(player['name'])
        total_variance += get_variance(player['name'])
    
    all_player_names = [p['name'] for p in match['players']]
    all_player_elos = [get_elo(name) for name in all_player_names]
    game_avg_elo = sum(all_player_elos) / len(all_player_elos)

    winner_win_prob.append(1 / (1 + math.exp(-expected_score / 2 if match['cap_diff'] > 0 else expected_score / 2)))
    pred_errors.append(match['cap_diff'] - min(max(expected_score, -5), 5))
    error = DIFF_MAPPING[match['cap_diff']] -\
        match['garbage_time_cap_diff'] * GARBAGE_TIME_DISCOUNT -\
        min(max(expected_score, DIFF_MAPPING[-5]), DIFF_MAPPING[5])
    
    red_stats_initial, blue_stats_initial = judge_stats_initial(match['players'])
    red_stats_converged, blue_stats_converged = judge_stats_converged(match['players'])
    for player, stat_score_initial, stat_score_converged in zip(match['players'][:4], red_stats_initial, red_stats_converged):
        elo_vs_game_avg = get_elo(player['name']) - game_avg_elo
        update_rating(player['name'], error, stat_score_initial, stat_score_converged, total_variance, elo_vs_game_avg)
    for player, stat_score_initial, stat_score_converged in zip(match['players'][4:], blue_stats_initial, blue_stats_converged):
        elo_vs_game_avg = get_elo(player['name']) - game_avg_elo
        update_rating(player['name'], -error, stat_score_initial, stat_score_converged, total_variance, elo_vs_game_avg)

In [144]:
elo = {}
variance = {}
pred_errors = []
winner_win_prob = []
day = 0

for match in matches:
    new_day = round(match['timestamp'] / 86400)
    if new_day != day:
        normalize_elos()
    day = new_day
    update_ratings(match)

print(f"MAE: {sum([abs(e) for e in pred_errors]) / len(pred_errors):.4f}    MSE: {sum([e ** 2 for e in pred_errors]) / len(pred_errors):.4f}")
print(f"COR: {sum([(1 if x > 0.5 else (0.5 if x == 0.5 else 0)) for x in winner_win_prob]) / len(winner_win_prob):.2%}    WWP: {sum(winner_win_prob) / len(winner_win_prob):.2%}")
print(f"BRI: {1 - (sum([(1 - x) ** 2 for x in winner_win_prob]) / len(winner_win_prob)) ** 0.5:.2%}    LOG: {2 ** (sum([math.log2(x) for x in winner_win_prob]) / len(winner_win_prob)):.2%}")

MAE: 2.3820    MSE: 8.2718
COR: 59.70%    WWP: 52.70%
BRI: 51.54%    LOG: 51.60%


In [145]:
print(f"MAE: {sum([abs(e) for e in pred_errors[10000:]]) / len(pred_errors[10000:]):.4f}    MSE: {sum([e ** 2 for e in pred_errors[10000:]]) / len(pred_errors[10000:]):.4f}")
print(f"COR: {sum([(1 if x > 0.5 else (0.5 if x == 0.5 else 0)) for x in winner_win_prob[10000:]]) / len(winner_win_prob[10000:]):.2%}    WWP: {sum(winner_win_prob[10000:]) / len(winner_win_prob[10000:]):.2%}")
print(f"BRI: {1 - (sum([(1 - x) ** 2 for x in winner_win_prob[10000:]]) / len(winner_win_prob[10000:])) ** 0.5:.2%}    LOG: {2 ** (sum([math.log2(x) for x in winner_win_prob[10000:]]) / len(winner_win_prob[10000:])):.2%}")

MAE: 2.3767    MSE: 8.2276
COR: 58.72%    WWP: 52.20%
BRI: 51.22%    LOG: 51.26%


In [146]:
leaderboard = sorted([
    p for p in elo if variance[p] <= 0.3
], key=lambda x: elo[x], reverse=True)
for i, player in enumerate(leaderboard):
    print(f"{i + 1:>3}.  {player:<12}  {elo[player]:5.2f} ± {1.96 * variance[player] ** 0.5:.2f}")

  1.  okthen         2.78 ± 0.56
  2.  DT             2.60 ± 0.69
  3.  toasty         2.52 ± 0.84
  4.  CarrotCake     2.50 ± 0.55
  5.  SluffAndRuff   2.42 ± 0.65
  6.  may_hem        2.38 ± 0.37
  7.  Alphachurro    2.30 ± 0.72
  8.  phreak         2.16 ± 0.61
  9.  meowza         2.14 ± 0.45
 10.  Ritual         2.13 ± 0.63
 11.  mex            2.10 ± 0.52
 12.  OuchMyBalls    2.09 ± 0.44
 13.  Shikari        2.09 ± 0.51
 14.  Xx360NoSwagx   2.07 ± 0.87
 15.  Suchit         2.04 ± 0.33
 16.  danp           2.03 ± 0.49
 17.  fender         1.99 ± 0.53
 18.  BALLDON'TLIE   1.97 ± 0.48
 19.  jig            1.92 ± 0.70
 20.  ft             1.81 ± 0.69
 21.  Messi          1.81 ± 0.63
 22.  realtea        1.75 ± 0.44
 23.  Enervate       1.70 ± 0.45
 24.  tits           1.69 ± 0.68
 25.  Maelstrom      1.69 ± 0.44
 26.  kool aid       1.68 ± 0.67
 27.  ASAP           1.65 ± 0.68
 28.  grunt          1.64 ± 0.75
 29.  Crippy         1.62 ± 0.71
 30.  Ball-E         1.61 ± 0.95
 31.  Dori

In [147]:
import random

ranks = []
for _ in range(10000):
    simulated_elos = {p: elo[p] + variance[p] ** 0.5 * random.gauss(0, 1) for p in elo if variance[p] <= 0.3}
    sorted_players = sorted(simulated_elos.items(), key=lambda x: x[1], reverse=True)
    ranks.append({player: rank + 1 for rank, (player, _) in enumerate(sorted_players)})

for i, player in enumerate(leaderboard[:]):
    rank_ci = sorted([r[player] for r in ranks])[500:-500]
    print(f"{i + 1:>3}.  {player:<12}  {elo[player]:5.2f} ± {1.96 * variance[player] ** 0.5:.2f}  ({min(rank_ci)}-{max(rank_ci)})")

  1.  okthen         2.78 ± 0.56  (1-9)
  2.  DT             2.60 ± 0.69  (1-18)
  3.  toasty         2.52 ± 0.84  (1-26)
  4.  CarrotCake     2.50 ± 0.55  (1-17)
  5.  SluffAndRuff   2.42 ± 0.65  (1-23)
  6.  may_hem        2.38 ± 0.37  (3-16)
  7.  Alphachurro    2.30 ± 0.72  (1-33)
  8.  phreak         2.16 ± 0.61  (3-35)
  9.  meowza         2.14 ± 0.45  (4-28)
 10.  Ritual         2.13 ± 0.63  (3-40)
 11.  mex            2.10 ± 0.52  (4-34)
 12.  OuchMyBalls    2.09 ± 0.44  (6-31)
 13.  Shikari        2.09 ± 0.51  (4-34)
 14.  Xx360NoSwagx   2.07 ± 0.87  (2-61)
 15.  Suchit         2.04 ± 0.33  (8-29)
 16.  danp           2.03 ± 0.49  (6-37)
 17.  fender         1.99 ± 0.53  (6-42)
 18.  BALLDON'TLIE   1.97 ± 0.48  (7-42)
 19.  jig            1.92 ± 0.70  (5-61)
 20.  ft             1.81 ± 0.69  (7-69)
 21.  Messi          1.81 ± 0.63  (8-65)
 22.  realtea        1.75 ± 0.44  (14-56)
 23.  Enervate       1.70 ± 0.45  (15-61)
 24.  tits           1.69 ± 0.68  (10-82)
 25.  Maelstro